In [1]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

import tensorflow as tf
# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa

from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd

%matplotlib inline

In [2]:
learning_rate = 0.01
seq_length = 7 #layer
# train Parameters
X_dim = 13 #n_dim #X_hot.shape[2]

#seq_length = 7 #X_hot.shape[1]
output_dim = 2 #n_classes #Y_hot.shape[1]

hidden_dim = 2
learning_rate = 0.05

In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, X_dim], name = 'X')
Y = tf.placeholder(tf.float32, [None, output_dim], name = 'Y')


# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Y_pred, labels = Y))
# optimizer
lr = tf.placeholder(tf.float32,shape=(), name='learning_rate')
train = tf.train.AdamOptimizer(lr).minimize(loss) #AdamOptimizer

# RMSE
targets = tf.placeholder(tf.float32, [None, output_dim] , name = 'targets')
predictions = tf.placeholder(tf.float32, [None, output_dim] , name = 'predictions')
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))


In [17]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 0.2
WAVE_OUTPUT_FILENAME = "../data/output.wav"

p = pyaudio.PyAudio()

stream = p.open(format = FORMAT,channels = CHANNELS,rate = RATE,input = True,frames_per_buffer = CHUNK)

print("start to record the audio.")

frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
    
print("Recording finished.")
stream.stop_stream()
stream.close()

p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

start to record the audio.
Recording finished.


In [19]:
import glob
model_path_f = '../models/RNN/'
filename = 'my_RNN_model_500.meta'

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
loader = tf.train.import_meta_graph(model_path_f+filename)
loader.restore(sess, tf.train.latest_checkpoint(model_path_f))

SR = RATE
sound_path = WAVE_OUTPUT_FILENAME
sample, sample_rate = librosa.load(sound_path,SR)
print(sample.shape)

S = librosa.feature.melspectrogram(sample, sr=sample_rate, n_mels=128)
# Convert to log scale (dB). We'll use the peak power (max) as reference.
log_S = librosa.power_to_db(S, ref=np.max)
mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)

# Let's pad on the first and second deltas while we're at it
delta2_mfcc = librosa.feature.delta(mfcc, order=2)
print(delta2_mfcc.shape , sample_rate)

dataX = delta2_mfcc.T
print('delta shape:',dataX.shape)

X_input = np.array(dataX[dataX.shape[0] - seq_length:])
X_input = X_input.reshape(1,seq_length,13)
print('X_input shape:',X_input.shape)
print('\n\n\n')
y_pred = None
y_pred = sess.run(tf.argmax(Y_pred,1),feed_dict={X: X_input})
y_pred = sess.run(tf.argmax(Y_pred,1),feed_dict={X: X_input})
print(y_pred, Y_pred)
if y_pred[0] == 1:
    print('The sound is Drone')
else :
    print('THe sound isn\'t Drone')
sess.close()

c:\users\jh\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning:

An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).



INFO:tensorflow:Restoring parameters from ../models/my_RNN_model_500
(8192,)
(13, 17) 44100
delta shape: (17, 13)
X_input shape: (1, 7, 13)




[0] Tensor("fully_connected/BiasAdd:0", shape=(?, 2), dtype=float32)
THe sound isn't Drone


In [25]:

model_path_f = '../models/RNN/'
filename = 'my_RNN_model_500.meta'


sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
loader = tf.train.import_meta_graph(model_path_f+filename)
loader.restore(sess, tf.train.latest_checkpoint(model_path_f))

SR = 44100
sound_path = WAVE_OUTPUT_FILENAME
####
while(1):
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT,channels = CHANNELS,rate = RATE,input = True,frames_per_buffer = CHUNK)

    #print("start to record the audio.")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    #print("Recording finished.")
    stream.stop_stream()
    stream.close()

    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    ####

    sample, sample_rate = librosa.load(sound_path,SR)
    #print(sample.shape)

    S = librosa.feature.melspectrogram(sample, sr=sample_rate, n_mels=128)
    # Convert to log scale (dB). We'll use the peak power (max) as reference.
    log_S = librosa.power_to_db(S, ref=np.max)
    mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)

    # Let's pad on the first and second deltas while we're at it
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    print(delta2_mfcc.shape , sample_rate)

    dataX = delta2_mfcc.T
    print('delta shape:',dataX.shape)

    X_input = np.array(dataX[dataX.shape[0] - seq_length:])
    X_input = X_input.reshape(1,seq_length,13)
    #print('X_input shape:',X_input.shape)
    #print('\n\n\n')
    y_pred = None
    y_pred = sess.run(tf.argmax(Y_pred,1),feed_dict={X: X_input})

    if y_pred[0] == 1:
        print('The sound is Drone')
    else :
        print('THe sound isn\'t Drone')

sess.close()

c:\users\jh\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py:1711: UserWarning:

An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).



INFO:tensorflow:Restoring parameters from ../models/RNN/my_RNN_model_500
(13, 17) 44100
delta shape: (17, 13)
THe sound isn't Drone
(13, 17) 44100
delta shape: (17, 13)
THe sound isn't Drone
(13, 17) 44100
delta shape: (17, 13)
The sound is Drone
(13, 17) 44100
delta shape: (17, 13)
The sound is Drone
(13, 17) 44100
delta shape: (17, 13)


KeyboardInterrupt: 